In [42]:
from heapq import heappop, heappush, heapify
from tqdm import tqdm


def read(robot_nums):
    with open("input.txt") as f:
        instructions = [i.strip() for i in f.readlines()]
    dirs = [[["7", "8", "9"], ["4", "5", "6"], ["1", "2", "3"], ["X", "0", "A"]]] + [[["X", "^", "A"], ["<", "v", ">"]] for _ in range(robot_nums)]
    robots = [(3, 2)] + [(0, 2) for _ in range(robot_nums)]
    return instructions, dirs, robots


def dijkstra(board, robot_pos, end):
    heapify(heap := [(0, robot_pos)])
    dp = [[float("inf") for _ in range(len(board[0]))] for _ in range(len(board))]
    prev = [[set() for _ in range(len(board[0]))] for _ in range(len(board))]
    dp[robot_pos[0]][robot_pos[1]] = 0
    min_d = float("inf")
    while heap:
        d, (y, x) = heappop(heap)
        if d > min_d:
            continue
        if board[y][x] == end:
            end_coor = (y, x)
            min_d = d
            continue
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if 0 <= ny < len(board) and 0 <= nx < len(board[0]) and board[ny][nx] != "X":
                if dp[ny][nx] > d + 1:
                    dp[ny][nx] = d + 1
                    prev[ny][nx].add((y, x))
                    heappush(heap, (d + 1, (ny, nx)))
                elif dp[ny][nx] == d + 1:
                    prev[ny][nx].add((y, x))
    paths = []
    stack = [(end_coor, [])]
    while stack:
        (y, x), curr_path = stack.pop()
        if (y, x) == robot_pos:
            paths.append([(y, x)] + curr_path)
        else:
            for py, px in prev[y][x]:
                stack.append(((py, px), [(y, x)] + curr_path))
    unique_paths = list({tuple(path) for path in paths})
    movements = []
    for path in unique_paths:
        move = []
        start = path[0]
        for y, x in path[1:]:
            if y > start[0]:
                move.append("v")
            elif y < start[0]:
                move.append("^")
            elif x > start[1]:
                move.append(">")
            else:
                move.append("<")
            start = (y, x)
        move.append("A")
        movements.append("".join(move))
    return end_coor, movements


def helper(dirs, robots_pos, robots_end, idx, dp_helper):
    robot_start = robots_pos[idx]
    robot_next, moves_next = dijkstra(dirs[idx], robot_start, robots_end)
    robots_pos[idx] = robot_next
    min_robot_move = float("inf")
    for robot_move in moves_next:
        if (idx, robot_move) in dp_helper:
            temp_robot_move = dp_helper[(idx, robot_move)]
        else:
            temp_robot_move = 0
            for i in robot_move:
                if idx < len(dirs) - 1:
                    temp_robot_move += helper(dirs, robots_pos, i, idx + 1, dp_helper)
                else:
                    temp_robot_move += 1
            dp_helper[(idx, robot_move)] = temp_robot_move
        min_robot_move = min(min_robot_move, temp_robot_move)
    return min_robot_move


def solver(robot_nums):
    instructions, dirs, robots = read(robot_nums)
    complexities = 0
    dp_helper = dict()
    for ins in tqdm(instructions):
        shortest_path = 0
        for i in ins:
            shortest_path += helper(dirs, robots, i, 0, dp_helper)
        complexities += int(ins[:-1]) * shortest_path
    print(complexities)

In [43]:
# Q1
solver(2)

100%|██████████| 5/5 [00:00<00:00, 1792.13it/s]

157908


In [44]:
# Q2
solver(25)

100%|██████████| 5/5 [00:00<00:00, 289.23it/s]

196910339808654
